In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv()
import pickle as pk
import numpy as np
from pathlib import Path
base_dir = os.getenv('BASEDIR')

from tqdm import tqdm
tqdm.pandas()

import scattertext as st
import re

import spacy

2023-03-23 16:03:34.782741: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-23 16:03:42.197314: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-23 16:03:44.680534: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-23 16:03:44.680553: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
dataset='qanda'

In [3]:
dataset_name='qanda'
data = pd.read_csv(os.path.join(base_dir,'data','01_raw_data','qanda','qanda_emoji.csv'), dtype=str, names=['tid', 'rid' ,'qid', 'uid','name', 'bio', 'date', 'text_extended', 'text', 'hashtags', 'hashtags_extended', 'mentions', 'mentions_extended', 'urls_extended', 'urls'], index_col=False)

data['text'] = data['text'].fillna('').apply(lambda t: re.sub(r'http\S+', '', t)).apply(lambda t: re.sub("#[A-Za-z0-9_]+","", t)).apply(lambda t: re.sub(r'@\S+', '', t))
data['bio'] = data['bio'].fillna('').apply(lambda t: re.sub(r'http\S+', '', t)).apply(lambda t: re.sub("#[A-Za-z0-9_]+","", t)).apply(lambda t: re.sub(r'@\S+', '', t))
data['name'] = data['name'].fillna('')

In [4]:
# data = data[(data['rid'].isna() | (data['rid'] == '0'))]

In [5]:
data_per_user = data.groupby('uid').progress_apply(lambda d: pd.DataFrame({'name' : [d['name'].values[0]], 'bio' : [d['bio'].values[0]], 'text' : [' '.join(d['text'])] }))

  0%|          | 0/103074 [00:00<?, ?it/s]

In [6]:
data_per_user = data_per_user.reset_index(drop=False).set_index('uid')[['name','bio','text']]

In [7]:
inpath = os.path.join(base_dir,'data','04_results','labels',dataset+'_extended.csv')

In [8]:
labels_df = pd.read_csv(inpath, dtype=str).set_index('uid')[['lr_label', 'fr_label']]

In [9]:
mask = labels_df.fr_label.map({'True':True, 'False':False})

In [10]:
labels_df['fr_label'][mask] = 'FR'
labels_df['fr_label'][~mask] = 'M'

In [11]:
df = pd.merge(data_per_user,labels_df, left_index=True, right_index=True, how='left')

In [12]:
# category_col = 'lr_label'
# category = 'L'
# category_name='Left'
# not_category_name='Right'

category_col = 'fr_label'
category = 'M'
category_name='Moderate'
not_category_name='Far-Right'

width=500

In [13]:
if category_col == 'lr_label':
    # sampled_df = df[df.lr_label != 'N'].groupby('lr_label').sample(n=700, random_state=123)
    sampled_df = df[df.lr_label != 'N']
else:
    # sampled_df = df.groupby('fr_label').sample(n=660, random_state=123)
    sampled_df = df

In [14]:
sampled_df['parse'] = sampled_df.text.progress_apply(st.whitespace_nlp_with_sentences)

  0%|          | 0/103074 [00:00<?, ?it/s]

In [20]:
st.CohensD(corpus).use_metadata().set_categories(
    'M', ['FR']
)

In [21]:
corpus

In [15]:
corpus = st.OffsetCorpusFactory(
    sampled_df,
    category_col=category_col,
    parsed_col='parse',
    feat_and_offset_getter=st.cognitive_distortions_offset_getter_factory()
).build()

plot_df = st.CohensD(corpus).use_metadata().set_categories(
    'M', ['FR']
).get_score_df(
).assign(
    N = lambda df: df.count1+df.count2,
    X=lambda df: df.N,
    Y=lambda df: df.hedges_r,
    Xpos=lambda df: st.Scalers.dense_rank(df.X),
    Ypos=lambda df: st.Scalers.scale_center_zero_abs(df.Y),
    SuppressDisplay=False,
    ColorScore=lambda df: df.Ypos,
)


/data/roram/anaconda3/envs/rr/lib/python3.10/site-packages/scipy/sparse/_base.py:665: RuntimeWarning: invalid value encountered in divide
  return np.true_divide(self.todense(), other)


In [17]:
plot_df.columns

Index(['cohens_d', 'cohens_d_se', 'cohens_d_z', 'cohens_d_p', 'hedges_r',
       'hedges_r_se', 'hedges_r_z', 'hedges_r_p', 'm1', 'm2', 'count1',
       'count2', 'docs1', 'docs2', 'N', 'X', 'Y', 'Xpos', 'Ypos',
       'SuppressDisplay', 'ColorScore'],
      dtype='object')

In [18]:
plot_df

,cohens_d,cohens_d_se,cohens_d_z,cohens_d_p,hedges_r,hedges_r_se,hedges_r_z,hedges_r_p,m1,m2,...,count2,docs1,docs2,N,X,Y,Xpos,Ypos,SuppressDisplay,ColorScore
term,,,,,,,,,,,,,,,,,,,,,
Dichotomous Reasoning,0.220006,0.006248,35.210205,6.978361e-272,0.220004,0.036728,5.990126,1.048393e-09,0.232055,0.147980,...,4925,30257,152,216044,216044,0.220004,1.000000,1.000000,False,1.000000
Should statements,0.199330,0.006245,31.918468,7.401294e-224,0.199329,0.036727,5.427364,2.859623e-08,0.088431,0.041165,...,1352,17184,104,80756,80756,0.199329,0.909091,0.953012,False,0.953012
Labeling and mislabeling,0.023532,0.006230,3.777418,7.923144e-05,0.023532,0.036721,0.640829,2.608168e-01,0.014273,0.012047,...,267,4907,69,13557,13557,0.023532,0.727273,0.553481,False,0.553481
Mindreading,0.112183,0.006234,17.994142,1.082747e-72,0.112182,0.036723,3.054822,1.125969e-03,0.022980,0.009071,...,403,6622,58,19085,19085,0.112182,0.818182,0.754955,False,0.754955
Personalizing,0.017921,0.006230,2.876753,2.008949e-03,0.017921,0.036721,0.488027,3.127653e-01,0.001918,0.001284,...,34,769,15,1330,1330,0.017921,0.454545,0.540729,False,0.540729
Overgeneralizing,-0.007018,0.006230,-1.126589,8.700419e-01,-0.007018,0.036721,-0.191117,5.757832e-01,0.006284,0.006735,...,65,2475,35,6027,6027,-0.007018,0.545455,0.484050,False,0.484050
Magnification and Minimization,0.058158,0.006231,9.333802,5.107007e-21,0.058157,0.036722,1.583719,5.662880e-02,0.009786,0.005079,...,173,3277,39,8325,8325,0.058157,0.636364,0.632173,False,0.632173
Mental Filtering,0.000894,0.006230,0.143485,4.429535e-01,0.000894,0.036721,0.024341,4.902903e-01,0.000256,0.000245,...,6,111,4,171,171,0.000894,0.090909,0.502031,False,0.502031
Catastrophizing,0.044811,0.006230,7.192397,3.183186e-13,0.044811,0.036722,1.220276,1.111801e-01,0.001699,0.000101,...,8,600,8,1119,1119,0.044811,0.363636,0.601840,False,0.601840


In [150]:
html = st.dataframe_scattertext(
    corpus,
    plot_df=plot_df,
    category=category,
    category_name=category_name,
    not_category_name=not_category_name,
    width_in_pixels=width,
    suppress_text_column='Display',
    metadata=corpus.get_df().index,
    use_non_text_features=True,
    ignore_categories=False,
    use_offsets=True,
    unified_context=False,
    color_score_column='ColorScore',
    left_list_column='ColorScore',
    y_label='Hedges R',
    x_label='Frequency Ranks',
    header_names={'upper': 'Top M', 'lower': 'Top FR'},
    save_svg_button=True,
)

/data/roram/anaconda3/envs/rr/lib/python3.10/site-packages/scattertext/Scalers.py:247: RuntimeWarning: invalid value encountered in divide
  vec_ss = (vec_ss - vec_ss.min()) * 1. / (vec_ss.max() - vec_ss.min())


In [151]:
open('./html/'+category_col+'/CDS_scattertext.html', 'w').write(html)

148805584

In [ ]:
import scattertext as st

moral_foundations_feats = st.FeatsFromMoralFoundationsDictionary()

corpus = st.CorpusFromPandas(sampled_df,
                             category_col=category_col,
                             text_col='text',
                             nlp=st.whitespace_nlp_with_sentences,
                             feats_from_spacy_doc=moral_foundations_feats).build()

html = st.produce_frequency_explorer(
    corpus,
    category=category,
    category_name=category_name,
    not_category_name=not_category_name,
    # metadata=sampled_df.index,
    use_non_text_features=True,
    use_full_doc=True,
    term_scorer=st.CohensD(corpus).use_metadata(),
    grey_threshold=0,
    width_in_pixels=width,
    topic_model_term_lists=moral_foundations_feats.get_top_model_term_lists(),                
    metadata_descriptions=moral_foundations_feats.get_definitions(),
    # return_scatterplot_structure=True,
    save_svg_button=True,
)

open('./html/'+category_col+'/mft_scattertext.html', 'w').write(html)

In [ ]:
sampled_df['text'] = sampled_df['text'].apply(lambda t: t[:1000000])
# Work-around for spacy

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
corpus = st.CorpusFromPandas(sampled_df,
                             category_col=category_col,
                             text_col='text',
                             nlp=nlp).build()

In [ ]:
html = st.produce_scattertext_explorer(corpus,
    category=category,
    category_name=category_name,
    not_category_name=not_category_name,
    # metadata=sampled_df.index,
    width_in_pixels=width,
    save_svg_button=True,
)

In [ ]:
open('./html/'+category_col+'/term_scattertext.html', 'w').write(html)

In [ ]:
# import pytextrank, spacy
# import scattertext as st

# nlp = spacy.load('en_core_web_sm')
# nlp.add_pipe("textrank", last=True)

# sampled_df['parse'] = sampled_df.text.progress_apply(nlp)

# corpus = st.CorpusFromParsedDocuments(
#     sampled_df,
#     category_col=category_col,
#     parsed_col='parse',
#     feats_from_spacy_doc=st.PyTextRankPhrases()
# ).build(
# ).compact(
#     st.AssociationCompactor(2000, use_non_text_features=True)
# )

# term_category_scores = corpus.get_metadata_freq_df('')

# term_ranks = np.argsort(np.argsort(-term_category_scores, axis=0), axis=0) + 1

# category_specific_prominence = term_category_scores.apply(
#     lambda r: r.L if r.L > r.R else -r.R,
#     axis=1
# )

# html = st.produce_scattertext_explorer(
#     corpus,
#     category='L',
#     not_category_name='R',
#     minimum_term_frequency=0,
#     pmi_threshold_coefficient=0,
#     width_in_pixels=1000,
#     transform=st.dense_rank,
#     metadata=corpus.get_df().index,
#     scores=category_specific_prominence,
#     sort_by_dist=False,
#     use_non_text_features=True,
#     topic_model_term_lists={term: [term] for term in corpus.get_metadata()},
#     topic_model_preview_size=0,
#     # metadata_descriptions=metadata_descriptions,
#     use_full_doc=True,
#     save_svg_button=True,
# )

# open('./phrase_scattertext.html', 'w').write(html)

In [ ]:
import nltk

import scattertext as st
from scattertext.termranking import OncePerDocFrequencyRanker

In [ ]:
nlp = st.tweet_tokenizier_factory(nltk.tokenize.TweetTokenizer())
sampled_df['parse'] = (sampled_df.name + sampled_df.bio + sampled_df.text).progress_apply(nlp)

In [ ]:
corpus = st.CorpusFromParsedDocuments(
    sampled_df,
    category_col=category_col,
    parsed_col='parse',
    feats_from_spacy_doc=st.FeatsFromSpacyDocOnlyEmoji()
).build()

In [ ]:
html = st.produce_scattertext_explorer(
    corpus,
    category=category,
    category_name=category_name,
    not_category_name=not_category_name,
    use_full_doc=True,
    term_ranker=OncePerDocFrequencyRanker,
    sort_by_dist=False,
    metadata=sampled_df.index,
    width_in_pixels=width,
    save_svg_button=True,
)

In [ ]:
open('./html/'+category_col+'/emoji_scattertext_v2.html', 'w').write(html)

In [105]:
feat_builder = st.FeatsFromOnlyEmpath()

In [106]:
corpus = st.CorpusFromParsedDocuments(
    sampled_df,
    # df[df.lr_label != 'N'],
    category_col=category_col,
    parsed_col='text',
    feats_from_spacy_doc=feat_builder
).build()

In [107]:
html = st.produce_scattertext_explorer(
    corpus,
    category=category,
    category_name=category_name,
    not_category_name=not_category_name,
    width_in_pixels=width,
    metadata=sampled_df.index,
    use_non_text_features=True,
    use_full_doc=True,
    topic_model_term_lists=feat_builder.get_top_model_term_lists(),
    save_svg_button=True,
)

In [108]:
open('./html/'+category_col+'/empath_scattertext.html', 'w').write(html)

158625488

In [148]:
sampled_df['parse'] = sampled_df.text.progress_apply(st.whitespace_nlp_with_sentences)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 103074/103074 [05:00<00:00, 343.25it/s]


In [149]:
corpus = st.OffsetCorpusFactory(
    sampled_df,
    category_col=category_col,
    parsed_col='parse',
    feat_and_offset_getter=st.cognitive_distortions_offset_getter_factory()
).build()

plot_df = st.CohensD(corpus).use_metadata().set_categories(
    'M', ['FR']
).get_score_df(
).assign(
    N = lambda df: df.count1+df.count2,
    X=lambda df: df.N,
    Y=lambda df: df.hedges_r,
    Xpos=lambda df: st.Scalers.dense_rank(df.X),
    Ypos=lambda df: st.Scalers.scale_center_zero_abs(df.Y),
    SuppressDisplay=False,
    ColorScore=lambda df: df.Ypos,
)


/data/roram/anaconda3/envs/rr/lib/python3.10/site-packages/scipy/sparse/_base.py:665: RuntimeWarning: invalid value encountered in divide
  return np.true_divide(self.todense(), other)


In [1]:
plot_df

NameError: name 'plot_df' is not defined

In [150]:
html = st.dataframe_scattertext(
    corpus,
    plot_df=plot_df,
    category=category,
    category_name=category_name,
    not_category_name=not_category_name,
    width_in_pixels=width,
    suppress_text_column='Display',
    metadata=corpus.get_df().index,
    use_non_text_features=True,
    ignore_categories=False,
    use_offsets=True,
    unified_context=False,
    color_score_column='ColorScore',
    left_list_column='ColorScore',
    y_label='Hedges R',
    x_label='Frequency Ranks',
    header_names={'upper': 'Top M', 'lower': 'Top FR'},
    save_svg_button=True,
)

/data/roram/anaconda3/envs/rr/lib/python3.10/site-packages/scattertext/Scalers.py:247: RuntimeWarning: invalid value encountered in divide
  vec_ss = (vec_ss - vec_ss.min()) * 1. / (vec_ss.max() - vec_ss.min())


In [151]:
open('./html/'+category_col+'/CDS_scattertext.html', 'w').write(html)

148805584

In [115]:
import nltk

import scattertext as st
from scattertext.termranking import OncePerDocFrequencyRanker

In [116]:
nlp = st.tweet_tokenizier_factory(nltk.tokenize.TweetTokenizer())
sampled_df['parse'] = (sampled_df.name + sampled_df.bio + sampled_df.text).progress_apply(nlp)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 81416/81416 [02:42<00:00, 502.23it/s]
/tmp/ipykernel_252262/1131952529.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_df['parse'] = (sampled_df.name + sampled_df.bio + sampled_df.text).progress_apply(nlp)


In [117]:
corpus = st.CorpusFromParsedDocuments(
    sampled_df,
    category_col=category_col,
    parsed_col='parse',
    feats_from_spacy_doc=st.FeatsFromSpacyDocOnlyEmoji()
).build()

In [118]:
from scipy.stats import rankdata, hmean, norm

In [119]:
term_freq_df = corpus.get_term_freq_df()

In [120]:
term_freq_df

,L freq,R freq
term,,
🤔,5873,186
💬,131,0
🔥,1934,25
💁🏼‍♂️,1,0
😆,914,12
...,...,...
⛸️,1,0
💇‍♀️,1,0
💇‍♂️,1,0


In [121]:
primary, secondary = 'L', 'R'
primary_freq_name = primary+' freq'
secondary_freq_name = secondary+' freq'

term_freq_df[primary+'_precision'] = term_freq_df[primary_freq_name] * 1./(term_freq_df[primary_freq_name] + term_freq_df[secondary_freq_name])
term_freq_df[primary+'_freq_pct'] = term_freq_df[primary_freq_name] * 1./term_freq_df[primary_freq_name].sum()
term_freq_df[primary+'_hmean'] = term_freq_df.apply(lambda x: (hmean([x[primary+'_precision'], x[primary+'_freq_pct']])
                                                                   if x[primary+'_precision'] > 0 and x[primary+'_freq_pct'] > 0 
                                                                   else 0), axis=1)                                                        
term_freq_df.sort_values(by=primary+'_hmean', ascending=False).iloc[:10]

,L freq,R freq,L_precision,L_freq_pct,L_hmean
term,,,,,
👏,11830,179,0.985095,0.066527,0.124637
🤣,8958,336,0.963848,0.050376,0.095748
🐝,7422,74,0.990128,0.041738,0.080100
👇,7074,145,0.979914,0.039781,0.076458
🤔,5873,186,0.969302,0.033027,0.063878
😂,5307,364,0.935814,0.029844,0.057844
🇦🇺,3321,67,0.980224,0.018676,0.036653
🙄,3222,93,0.971946,0.018119,0.035575
💧,2989,111,0.964194,0.016809,0.033042


In [122]:
def scale(ar): 
    return (ar - ar.min()) / (ar.max() - ar.min())

def zero_centered_scale(ar):
    scores = np.zeros(len(ar))
    scores[ar > 0] = scale(ar[ar > 0])
    scores[ar < 0] = -scale(-ar[ar < 0])
    return (scores + 1) / 2.

frequencies_scaled = scale(np.log(term_freq_df.sum(axis=1).values))

In [123]:
from sklearn.linear_model import LogisticRegression
scores = corpus.get_logreg_coefs(category,
                                 LogisticRegression(penalty='l2', C=10, max_iter=10000, n_jobs=-1))
scores_scaled = zero_centered_scale(scores)


In [124]:
html = st.produce_scattertext_explorer(corpus,
                                    category=category,
                                    category_name=category_name,
                                    not_category_name=not_category_name,
                                    minimum_term_frequency=5,
                                    width_in_pixels=width,
                                    x_coords=frequencies_scaled,
                                    y_coords=scores_scaled,
                                    scores=scores,
                                    sort_by_dist=False,
                                    metadata=sampled_df.index,
                                    x_label='Log frequency',
                                    y_label='L2-Penalized Log Reg Coef',
                                    save_svg_button=True,)
file_name = './html/'+category_col+'/emoji_scattertext_v3.html'
open(file_name, 'wb').write(html.encode('utf-8'))
# IFrame(src=file_name, width = 1200, height=700)

/data/roram/anaconda3/envs/rr/lib/python3.10/site-packages/scattertext/termscoring/ScaledFScore.py:303: RuntimeWarning: invalid value encountered in divide
  recall = cat_word_counts * 1. / cat_word_counts.sum()


141465067

In [125]:
html = st.produce_scattertext_explorer(corpus,
                                    category=category,
                                    category_name=category_name,
                                    not_category_name=not_category_name,
                                    minimum_term_frequency=5,
                                    width_in_pixels=width,
                                    x_coords=frequencies_scaled,
                                    y_coords=corpus.get_scaled_f_scores(category, beta=0.5),
                                    scores=corpus.get_scaled_f_scores(category, beta=0.5),
                                    sort_by_dist=False,
                                    metadata=sampled_df.index,
                                    x_label='Log Frequency',
                                    y_label='Scaled F-Score',
                                    save_svg_button=True,)
file_name = './html/'+category_col+'/emoji_scattertext_v4.html'
open(file_name, 'wb').write(html.encode('utf-8'))
# IFrame(src=file_name, width = 1200, height=700)

141464238

In [126]:
freq_df = corpus.get_term_freq_df().rename(columns={'L freq': 'y_dem', 'R freq': 'y_rep'})
a_w = 0.01
y_i, y_j = freq_df['y_dem'].values, freq_df['y_rep'].values

In [127]:
n_i, n_j = y_i.sum(), y_j.sum()
a_0 = len(freq_df) * a_w
delta_i_j = (  np.log((y_i + a_w) / (n_i + a_0 - y_i - a_w))
                 - np.log((y_j + a_w) / (n_j + a_0 - y_j - a_w)))
var_delta_i_j = ( 1./(y_i + a_w) + 1./(y_i + a_0 - y_i - a_w)
                    + 1./(y_j + a_w) + 1./(n_j + a_0 - n_j - a_w))
zeta_i_j = delta_i_j/np.sqrt(var_delta_i_j)
max_abs_zeta = max(zeta_i_j.max(), -zeta_i_j.min())
zeta_scaled_for_charting = ((((zeta_i_j > 0).astype(float) * (zeta_i_j/max_abs_zeta))*0.5 + 0.5)
                            + ((zeta_i_j < 0).astype(float) * (zeta_i_j/max_abs_zeta) * 0.5))

In [128]:
html = st.produce_scattertext_explorer(corpus,
                                    category=category,
                                    category_name=category_name,
                                    not_category_name=not_category_name,
                                    minimum_term_frequency=5,
                                    width_in_pixels=width,
                                    x_coords=frequencies_scaled,
                                    y_coords=zeta_scaled_for_charting,
                                    scores=zeta_i_j,
                                    sort_by_dist=False,
                                    metadata=sampled_df.index,
                                    x_label='Log Frequency',
                                    y_label='Log Odds Ratio w/ Uninformative Prior (alpha_w=0.01)',
                                    save_svg_button=True,)
file_name = './html/'+category_col+'/emoji_scattertext_v5.html'
open(file_name, 'wb').write(html.encode('utf-8'))

141464861

In [129]:
corner_scores = corpus.get_corner_scores(category)
html = st.produce_scattertext_explorer(corpus,
                                    category=category,
                                    category_name=category_name,
                                    not_category_name=not_category_name,
                                    minimum_term_frequency=5,
                                    width_in_pixels=width,
                                    x_coords=frequencies_scaled,
                                    y_coords=corner_scores,
                                    scores=corner_scores,
                                    sort_by_dist=False,
                                    metadata=sampled_df.index,
                                    x_label='Log Frequency',
                                    y_label='Corner Scores',
                                    save_svg_button=True,)
file_name = './html/'+category_col+'/emoji_scattertext_v6.html'
open(file_name, 'wb').write(html.encode('utf-8'))

141464772

In [40]:
feat_builder = st.FeatsFromOnlyEmpath()

In [41]:
corpus = st.CorpusFromParsedDocuments(
    sampled_df,
    # df[df.lr_label != 'N'],
    category_col=category_col,
    parsed_col='text',
    feats_from_spacy_doc=feat_builder
).build()

In [45]:
corpus

In [42]:
from scipy.stats import rankdata, hmean, norm

In [43]:
term_freq_df = corpus.get_term_freq_df()

In [44]:
term_freq_df

,L freq,R freq
term,,


In [36]:
primary, secondary = 'L', 'R'
primary_freq_name = primary+' freq'
secondary_freq_name = secondary+' freq'

term_freq_df[primary+'_precision'] = term_freq_df[primary_freq_name] * 1./(term_freq_df[primary_freq_name] + term_freq_df[secondary_freq_name])
term_freq_df[primary+'_freq_pct'] = term_freq_df[primary_freq_name] * 1./term_freq_df[primary_freq_name].sum()
term_freq_df[primary+'_hmean'] = term_freq_df.apply(lambda x: (hmean([x[primary+'_precision'], x[primary+'_freq_pct']])
                                                                   if x[primary+'_precision'] > 0 and x[primary+'_freq_pct'] > 0 
                                                                   else 0), axis=1)                                                        
term_freq_df.sort_values(by=primary+'_hmean', ascending=False).iloc[:10]

ValueError: Wrong number of items passed 4, placement implies 1

In [ ]:
def scale(ar): 
    return (ar - ar.min()) / (ar.max() - ar.min())

def zero_centered_scale(ar):
    scores = np.zeros(len(ar))
    scores[ar > 0] = scale(ar[ar > 0])
    scores[ar < 0] = -scale(-ar[ar < 0])
    return (scores + 1) / 2.

frequencies_scaled = scale(np.log(term_freq_df.sum(axis=1).values))

In [21]:
from sklearn.linear_model import LogisticRegression
scores = corpus.get_logreg_coefs(category,
                                 LogisticRegression(penalty='l2', C=10, max_iter=10000, n_jobs=-1))
scores_scaled = zero_centered_scale(scores)


In [23]:
html = st.produce_scattertext_explorer(corpus,
                                    category=category,
                                    category_name=category_name,
                                    not_category_name=not_category_name,
                                    minimum_term_frequency=5,
                                    width_in_pixels=width,
                                    x_coords=frequencies_scaled,
                                    y_coords=scores_scaled,
                                    scores=scores,
                                    sort_by_dist=False,
                                    metadata=sampled_df.index,
                                    x_label='Log frequency',
                                    y_label='L2-Penalized Log Reg Coef',
                                    save_svg_button=True,)
file_name = './html/'+category_col+'/empath_scattertext_v3.html'
open(file_name, 'wb').write(html.encode('utf-8'))
# IFrame(src=file_name, width = 1200, height=700)

/data/roram/anaconda3/envs/rr/lib/python3.10/site-packages/scattertext/termscoring/ScaledFScore.py:303: RuntimeWarning: invalid value encountered in divide
  recall = cat_word_counts * 1. / cat_word_counts.sum()


141465069

In [25]:
html = st.produce_scattertext_explorer(corpus,
                                    category=category,
                                    category_name=category_name,
                                    not_category_name=not_category_name,
                                    minimum_term_frequency=5,
                                    width_in_pixels=width,
                                    x_coords=frequencies_scaled,
                                    y_coords=corpus.get_scaled_f_scores(category, beta=0.5),
                                    scores=corpus.get_scaled_f_scores(category, beta=0.5),
                                    sort_by_dist=False,
                                    metadata=sampled_df.index,
                                    x_label='Log Frequency',
                                    y_label='Scaled F-Score',
                                    save_svg_button=True,
                                      )
file_name = './html/'+category_col+'/empath_scattertext_v4.html'
open(file_name, 'wb').write(html.encode('utf-8'))
# IFrame(src=file_name, width = 1200, height=700)

141464240